https://jira.x5.ru/browse/CVMUPG-1095

In [1]:
name = 'CVMUPG-1095_rusagro'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/cvm_upgrade/')
sys.path.append('/home/jovyan/cvm_upgrade/upgrade/')
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
from upgrade.post_analysis2.tools.groups_metrics_aggregation import GroupsMetricsAggregation
from upgrade.post_analysis2.tools.guests_metrics_preprocessing import GuestsMetricsPreprocessing
from upgrade.post_analysis2.tools.post_analysis_params import PostAnalysisParams
from upgrade.post_analysis2.tools.metric_names_mapper import map_col_names_to_russian
from upgrade.post_analysis2.tools.experiment_participants import ExperimentParticipants
from upgrade.post_analysis2.tools.experiment_params import ExperimentParams
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *
from logic_filters import *
from segmentation import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/08 14:43:33 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [4]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

### Выбираем гостей нужного юзкейса

In [5]:
usecase = ['cross', 'upgrade', 'ump']
dt = datetime.date.today()

In [6]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase))
                     .select('account_no', 'customer_rk')
                            )

In [7]:
customers_usecase.count() # 15 862 299

15862299

### 1 проверяем на доступность отобранных гостей в определенную дату

In [8]:
check_date = '2022-02-22'

In [13]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:07


In [14]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:03


In [15]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:04


In [16]:
seg_sms_1 = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [17]:
seg_sms_1.write.parquet('temp1095_1', mode='overwrite')
seg_sms_1 = spark.read.parquet('temp1095_1')

In [18]:
seg_sms_1.count() # 5 980 552

5980552

### 2 проверяем на доступность отобранных гостей в определенную дату

In [19]:
check_date = '2022-03-08'

In [20]:
seg_sms1 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[0], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:21


In [21]:
seg_sms2 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[1], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:10


In [22]:
seg_sms3 = (sms_channel_filters_glowbyte(spark=spark,
                                         guests=customers_usecase, 
                                         usecase_name=usecase[2], 
                                         check_date=check_date, 
                                         debug_mode=False)
                                    .select('customer_rk')
                                    .distinct()
           )

 Время выполнения: 0:00:16


In [23]:
seg_sms_2 = seg_sms1.union(seg_sms2).union(seg_sms3).distinct()

In [24]:
seg_sms_2.write.parquet('temp1095_2', mode='overwrite')
seg_sms_2 = spark.read.parquet('temp1095_2')

In [25]:
seg_sms_2.count() # 8 736 926

8736926

### Проверка на доступность плю в магазинах

Мск, ЦФО, СЗФО

In [26]:
macroregion_dk = ['MRDCT', 'MRDCN', 'MRDND']

In [27]:
shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
            #.filter(F.col('federal_subject_dk').isin([23, 93, 123, 193, 74, 174,
             #                                         774, 16, 116, 716, 71, 32]))
            .filter(F.col('macroregion_dk').isin(macroregion_dk))
            .selectExpr('store_id as plant')
            .distinct()
            #.toPandas()['plant']
            #.tolist()
            )

plants = (shops.select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

9902

### Heavy users

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [28]:
start_date = datetime.date(2021, 11, 3)
end_date = datetime.date(2022, 2, 2)

In [29]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [44]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [45]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms_1, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms_2, on='customer_rk', how='inner')

In [46]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [47]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [48]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [18]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
syntethic_category_id = [96,57,75,73,101,55,56,22]
# plu_brand_code = ['5727','M082','M083','5728']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [19]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [35]:
plu_codes_cat = [3017503,3195297,3444609,3444610,3495835,3503399,3693488,3695167,3982578,4102644,
4105769,4112954,4093072,4175547,4182900,4182902]

In [49]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [50]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [51]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [52]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 2)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [53]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 2)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [54]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [55]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [56]:
seg = seg1.union(seg2)

In [57]:
seg.write.parquet('temp1095_seg1', mode='overwrite')
seg = spark.read.parquet('temp1095_seg1')

In [58]:
seg.count()

600362

In [59]:
seg_pd1 = seg.toPandas()

In [60]:
seg_pd1_seg1 = seg_pd1[seg_pd1.group == '1']
seg_pd1_seg2 = seg_pd1[seg_pd1.group == '2']

In [61]:
print(len(seg_pd1_seg1))
print(len(seg_pd1_seg2))

244174
356188


In [341]:
seg_pd1_seg1 = seg_pd1_seg1.astype('str')
seg_pd1_seg2 = seg_pd1_seg2.astype('str')

In [344]:
cross11, upgrade11, ump11 = get_usecases(seg_pd1_seg1)
cross12, upgrade12, ump12 = get_usecases(seg_pd1_seg2)

In [346]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross11), len(upgrade11), len(ump11), 
              len(cross11)+len(upgrade11)+len(ump11)))
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross12), len(upgrade12), len(ump12), 
              len(cross12)+len(upgrade12)+len(ump12)))

cross: 77577, upgrade: 89108, ump: 77489 (all: 244174)
cross: 133387, upgrade: 144644, ump: 78157 (all: 356188)


22/02/08 21:23:27 ERROR cluster.YarnScheduler: Lost executor 1924 on mn-hd0176.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.
22/02/08 21:24:57 ERROR cluster.YarnScheduler: Lost executor 1925 on mn-hd0176.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.


### Конкуренты

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [62]:
start_date = datetime.date(2021, 11, 3)
end_date = datetime.date(2022, 2, 2)

In [63]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [64]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [65]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms_1, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms_2, on='customer_rk', how='inner')

In [66]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [67]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [68]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [69]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
syntethic_category_id = [75, 22]
plu_brand_code_not_in = ['O119', 'P927']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [70]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .filter(~F.col('plu_brand_code').isin(plu_brand_code_not_in))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [71]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [137]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [138]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [139]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [140]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 1)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [141]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [142]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [143]:
seg_2 = seg1.union(seg2)

In [144]:
seg_2.write.parquet('temp1095_seg2', mode='overwrite')
seg_2 = spark.read.parquet('temp1095_seg2')

In [145]:
seg_2.count()

423730

In [146]:
seg_pd2 = seg_2.toPandas()

In [147]:
seg_pd2_seg1 = seg_pd2[seg_pd2.group == '1']
seg_pd2_seg2 = seg_pd2[seg_pd2.group == '2']

In [148]:
print(len(seg_pd2_seg1))
print(len(seg_pd2_seg2))

171585
252145


### Гео2

Свердловская область

In [84]:
federal_subject_dk = [66]

In [85]:
shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
            .filter(F.col('federal_subject_dk').isin(federal_subject_dk))
#             .filter(F.col('macroregion_dk').isin(macroregion_dk))
            .selectExpr('store_id as plant')
            .distinct()
            #.toPandas()['plant']
            #.tolist()
            )

plants = (shops.select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

916

### Heavy users

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [86]:
start_date = datetime.date(2021, 11, 3)
end_date = datetime.date(2022, 2, 2)

In [87]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [88]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [89]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms_1, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms_2, on='customer_rk', how='inner')

In [90]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [91]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [92]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [93]:
plu_codes_cat = [2146174,3170510,3672055,4009123,4017707,4072721,4072722,4181490]

In [94]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [95]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [96]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [97]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 2)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [98]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 2)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [99]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [100]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [102]:
seg_3 = seg1.union(seg2)

In [103]:
seg_3.write.parquet('temp1095_seg3', mode='overwrite')
seg_3 = spark.read.parquet('temp1095_seg3')

[Stage 220:===================================================> (195 + 5) / 200]]]]]]0]]95]]22/02/08 17:48:16 ERROR client.TransportResponseHandler: Still have 1 requests outstanding when connection from /192.168.234.55:40444 is closed


In [104]:
seg_3.count()

164169

In [106]:
seg_pd3 = seg_3.toPandas()

In [107]:
seg_pd3_seg1 = seg_pd3[seg_pd3.group == '1']
seg_pd3_seg2 = seg_pd3[seg_pd3.group == '2']

In [108]:
print(len(seg_pd3_seg1))
print(len(seg_pd3_seg2))

66550
97619


### Конкуренты

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [109]:
start_date = datetime.date(2021, 11, 3)
end_date = datetime.date(2022, 2, 2)

In [110]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [111]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [112]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms_1, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms_2, on='customer_rk', how='inner')

In [113]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [114]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [115]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [116]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
syntethic_category_id = [75, 22]
plu_brand_code_not_in = ['O119', 'P927']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [117]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .filter(~F.col('plu_brand_code').isin(plu_brand_code_not_in))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [118]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [149]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [150]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [151]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [152]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 1)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [153]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [154]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [155]:
seg_4 = seg1.union(seg2)

In [156]:
seg_4.write.parquet('temp1095_seg4', mode='overwrite')
seg_4 = spark.read.parquet('temp1095_seg4')

In [157]:
seg_4.count()

423730

In [158]:
seg_pd4 = seg_4.toPandas()

In [159]:
seg_pd4_seg1 = seg_pd4[seg_pd4.group == '1']
seg_pd4_seg2 = seg_pd4[seg_pd4.group == '2']

In [160]:
print(len(seg_pd4_seg1))
print(len(seg_pd4_seg2))

171585
252145


### Гео3

Саратовская область

In [251]:
federal_subject_dk = [64]

In [252]:
shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
            .filter(F.col('federal_subject_dk').isin(federal_subject_dk))
#             .filter(F.col('macroregion_dk').isin(macroregion_dk))
            .selectExpr('store_id as plant')
            .distinct()
            #.toPandas()['plant']
            #.tolist()
            )

plants = (shops.select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

378

### Heavy users

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [253]:
start_date = datetime.date(2021, 11, 3)
end_date = datetime.date(2022, 2, 2)

In [254]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [255]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [256]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms_1, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms_2, on='customer_rk', how='inner')

In [257]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [258]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [259]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [260]:
plu_codes_cat = [3170888,3170890,3170891,3357823,3670061,4082170,4082171,4082172]

In [261]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [262]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [263]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [264]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 2)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [265]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 2)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [266]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [267]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [268]:
seg_5 = seg1.union(seg2)

In [269]:
seg_5.write.parquet('temp1095_seg5', mode='overwrite')
seg_5 = spark.read.parquet('temp1095_seg5')

In [270]:
seg_5.count()

32449

In [271]:
seg_pd5 = seg_5.toPandas()

In [272]:
seg_pd5_seg1 = seg_pd5[seg_pd5.group == '1']
seg_pd5_seg2 = seg_pd5[seg_pd5.group == '2']

In [273]:
print(len(seg_pd5_seg1))
print(len(seg_pd5_seg2))

13270
19179


### Конкуренты

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [274]:
start_date = datetime.date(2021, 11, 3)
end_date = datetime.date(2022, 2, 2)

In [275]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [276]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [277]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms_1, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms_2, on='customer_rk', how='inner')

In [278]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [279]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [280]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [281]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
syntethic_category_id = [75, 22]
plu_brand_code_not_in = ['O119', 'P927']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [282]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .filter(~F.col('plu_brand_code').isin(plu_brand_code_not_in))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [283]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [284]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [285]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [286]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [287]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 1)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [288]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [289]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [290]:
seg_6 = seg1.union(seg2)

In [291]:
seg_6.write.parquet('temp1095_seg6', mode='overwrite')
seg_6 = spark.read.parquet('temp1095_seg6')

In [292]:
seg_6.count()

175878

In [293]:
seg_pd6 = seg_6.toPandas()

In [294]:
seg_pd6_seg1 = seg_pd6[seg_pd6.group == '1']
seg_pd6_seg2 = seg_pd6[seg_pd6.group == '2']

In [295]:
print(len(seg_pd6_seg1))
print(len(seg_pd6_seg2))

71785
104093


### Гео4

Новосибирск

In [296]:
city_nm = ['Новосибирск г']

In [297]:
shops = (spark.table(DIM_STORE)
            .filter(F.col('valid_to_dttm')== datetime.datetime(5999, 1, 1, 0, 0))
#             .filter(F.col('federal_subject_dk').isin(federal_subject_dk))
#             .filter(F.col('macroregion_dk').isin(macroregion_dk))
            .filter(F.col('city_nm').isin(city_nm))
            .selectExpr('store_id as plant')
            .distinct()
            #.toPandas()['plant']
            #.tolist()
            )

plants = (shops.select('plant')
                .distinct()
                .toPandas()['plant']
                .tolist()
         )

len(plants)

204

### Heavy users

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [298]:
start_date = datetime.date(2021, 11, 3)
end_date = datetime.date(2022, 2, 2)

In [299]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [300]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [301]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms_1, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms_2, on='customer_rk', how='inner')

In [302]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [303]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [304]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [305]:
plu_codes_cat = [4085881,3606802,3606803,3623403,3623402,3606804]

In [306]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [307]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [308]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [309]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 2)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [310]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 2)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [311]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [312]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [313]:
seg_7 = seg1.union(seg2)

In [314]:
seg_7.write.parquet('temp1095_seg7', mode='overwrite')
seg_7 = spark.read.parquet('temp1095_seg7')

In [315]:
seg_7.count()

17688

In [316]:
seg_pd7 = seg_7.toPandas()

In [317]:
seg_pd7_seg1 = seg_pd7[seg_pd7.group == '1']
seg_pd7_seg2 = seg_pd7[seg_pd7.group == '2']

In [318]:
print(len(seg_pd7_seg1))
print(len(seg_pd7_seg2))

7264
10424


### Конкуренты

### Собираем чеки этих гостей и оставляем тех, кто покупал продукты с synthetic_catalog_id

In [319]:
start_date = datetime.date(2021, 11, 3)
end_date = datetime.date(2022, 2, 2)

In [320]:
checks_headers = (spark.table(CHECKS_HEADERS)
                      .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                      .filter((F.col('loyalty_card_no') != '') & (F.col('loyalty_card_no').isNotNull()))
                      .filter(F.col('financial_unit_format_dk') == 'D')
                      .filter(F.col('rtl_txn_cancel_flg') == 0)
                      .select('rtl_txn_id', 'loyalty_card_no', 'store_id')
                     )


In [321]:
checks_headers_tc5 = checks_headers.filter(F.col('store_id').isin(plants)) #оставили чеки только с нужными магазинами

In [322]:
loyalty_cards = (spark
                    .table(LOYALTY_CARDS)
                    .withColumnRenamed("loyalty_card_id", "loyalty_card_no")
                    .withColumnRenamed("loyalty_account_id", "account_no")
                    .withColumnRenamed("loyalty_account_acrm_id", "account_rk")
                    .select('account_no', 'loyalty_card_no')
                )
loyalty_cardholders = (spark
                        .table(LOYALTY_CARDHOLDERS)
                        .withColumnRenamed("loyalty_cardholder_acrm_id", "customer_rk")
                        .withColumnRenamed("loyalty_account_id", "account_no")
                        .select('account_no', 'customer_rk')
                      )
clients_info1 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info1 = clients_info1.join(seg_sms_1, on='customer_rk', how='inner')
clients_info2 = loyalty_cards.join(loyalty_cardholders, on='account_no', how='inner')
clients_info2 = clients_info2.join(seg_sms_2, on='customer_rk', how='inner')

In [323]:
checks_headers_tc51 = checks_headers_tc5.join(clients_info1, on='loyalty_card_no') #оставили чеки только нужных гостей

In [324]:
checks_headers_tc52 = checks_headers_tc5.join(clients_info2, on='loyalty_card_no') #оставили чеки только нужных гостей

In [325]:
checks_items = (spark.table(CHECKS_ITEMS) 
                    .withColumnRenamed('plu_id', 'plu_code')
                    .withColumnRenamed('turnover_no_vat_amt', 'zsalnovat')
                    .withColumnRenamed('turnover_vat_rub_amt', 'zsale_vat')
                    .withColumnRenamed('prime_cost_no_vat_amt', 'zcst_n')
                    .withColumnRenamed('turnover_base_uom_amt', 'base_qty')
                    .withColumnRenamed('discount_amt', 'zdiscount')
                    .withColumnRenamed('fact_regular_promo_flg', 'zpromofl')
                    .filter(F.col('rtl_txn_dt').between(start_date, end_date))
                    .filter((F.col('zsalnovat') >= 0) & (F.col('base_qty') >= 0) & (F.col('zcst_n') > 0)) #keep only correct data
                        .select('plu_code' #items id
                                , 'rtl_txn_id' #cheques id
                               , 'base_qty'
                               )
                   )


In [326]:
# plu_hierarchy_lvl_4_dk = ['FR0604001', 'FR0608005', 'FR0604002', 'FR0608001'] #сметана, молоко, кефир
syntethic_category_id = [75, 22]
plu_brand_code_not_in = ['O119', 'P927']
# plu_vendor_nm = ['Данон']
# plu_not_in = [4138521]

In [327]:
plu_codes_cat = (spark
                 .table(PRODUCTS)
#                  .filter(F.col('plu_hierarchy_lvl_4_dk').isin(plu_hierarchy_lvl_4_dk))
#                  .filter(F.col('plu_vendor_nm').isin(plu_vendor_nm))
                 .filter(F.col('syntethic_category_id').isin(syntethic_category_id))
                 .filter(~F.col('plu_brand_code').isin(plu_brand_code_not_in))
#                  .filter(~F.col('plu_id').isin(plu_not_in))
                 .select('plu_id')
                 .distinct()
                 .toPandas()['plu_id']
                 .tolist()
                )

In [328]:
checks_items = checks_items.filter(F.col('plu_code').isin(plu_codes_cat)) # только чеки с нужными plu

In [329]:
checks_tc51 = checks_items.join(checks_headers_tc51, 'rtl_txn_id', how='inner')

In [330]:
checks_tc52 = checks_items.join(checks_headers_tc52, 'rtl_txn_id', how='inner')

In [331]:
accs = ['account_no']
pdf = checks_tc51.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf = pdf.filter(F.col('base_qty_per_period') >= 1)
checks_tc51 = checks_tc51.join(pdf, on=accs)

In [332]:
pdf2 = checks_tc52.groupby(accs).agg(F.sum('base_qty').alias('base_qty_per_period'))
pdf2 = pdf2.filter(F.col('base_qty_per_period') >= 1)
checks_tc52 = checks_tc52.join(pdf2, on=accs)

In [333]:
seg1 = (checks_tc51.withColumn('group', F.lit('1'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [334]:
seg2 = (checks_tc52.withColumn('group', F.lit('2'))
                  .select('customer_rk', 'group')
                  .distinct()
      )

In [335]:
seg_8 = seg1.union(seg2)

In [336]:
seg_8.write.parquet('temp1095_seg8', mode='overwrite')
seg_8 = spark.read.parquet('temp1095_seg8')

In [337]:
seg_8.count()

107486

In [338]:
seg_pd8 = seg_8.toPandas()

In [339]:
seg_pd8_seg1 = seg_pd8[seg_pd8.group == '1']
seg_pd8_seg2 = seg_pd8[seg_pd8.group == '2']

In [340]:
print(len(seg_pd8_seg1))
print(len(seg_pd8_seg2))

43929
63557


In [347]:
seg_pd2_seg1 = seg_pd2_seg1.astype('str')
seg_pd2_seg2 = seg_pd2_seg2.astype('str')

In [348]:
cross21, upgrade21, ump21 = get_usecases(seg_pd2_seg1)
cross22, upgrade22, ump22 = get_usecases(seg_pd2_seg2)

In [349]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross21), len(upgrade21), len(ump21), 
              len(cross21)+len(upgrade21)+len(ump21)))
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross22), len(upgrade22), len(ump22), 
              len(cross22)+len(upgrade22)+len(ump22)))

cross: 56210, upgrade: 62058, ump: 53317 (all: 171585)
cross: 96360, upgrade: 101905, ump: 53880 (all: 252145)


In [350]:
seg_pd3_seg1 = seg_pd3_seg1.astype('str')
seg_pd3_seg2 = seg_pd3_seg2.astype('str')

In [351]:
cross31, upgrade31, ump31 = get_usecases(seg_pd3_seg1)
cross32, upgrade32, ump32 = get_usecases(seg_pd3_seg2)

In [352]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross31), len(upgrade31), len(ump31), 
              len(cross31)+len(upgrade31)+len(ump31)))
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross32), len(upgrade32), len(ump32), 
              len(cross32)+len(upgrade32)+len(ump32)))

cross: 21660, upgrade: 24101, ump: 20789 (all: 66550)
cross: 37008, upgrade: 39616, ump: 20995 (all: 97619)


In [353]:
seg_pd4_seg1 = seg_pd4_seg1.astype('str')
seg_pd4_seg2 = seg_pd4_seg2.astype('str')

In [354]:
cross41, upgrade41, ump41 = get_usecases(seg_pd4_seg1)
cross42, upgrade42, ump42 = get_usecases(seg_pd4_seg2)

In [355]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross41), len(upgrade41), len(ump41), 
              len(cross41)+len(upgrade41)+len(ump41)))
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross42), len(upgrade42), len(ump42), 
              len(cross42)+len(upgrade42)+len(ump42)))

cross: 56210, upgrade: 62058, ump: 53317 (all: 171585)
cross: 96360, upgrade: 101905, ump: 53880 (all: 252145)


In [356]:
seg_pd5_seg1 = seg_pd5_seg1.astype('str')
seg_pd5_seg2 = seg_pd5_seg2.astype('str')

In [357]:
cross51, upgrade51, ump51 = get_usecases(seg_pd5_seg1)
cross52, upgrade52, ump52 = get_usecases(seg_pd5_seg2)

In [358]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross51), len(upgrade51), len(ump51), 
              len(cross51)+len(upgrade51)+len(ump51)))
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross52), len(upgrade52), len(ump52), 
              len(cross52)+len(upgrade52)+len(ump52)))

cross: 4380, upgrade: 4750, ump: 4140 (all: 13270)
cross: 7356, upgrade: 7646, ump: 4177 (all: 19179)


In [359]:
seg_pd6_seg1 = seg_pd6_seg1.astype('str')
seg_pd6_seg2 = seg_pd6_seg2.astype('str')

In [360]:
cross61, upgrade61, ump61 = get_usecases(seg_pd6_seg1)
cross62, upgrade62, ump62 = get_usecases(seg_pd6_seg2)

In [361]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross61), len(upgrade61), len(ump61), 
              len(cross61)+len(upgrade61)+len(ump61)))
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross62), len(upgrade62), len(ump62), 
              len(cross62)+len(upgrade62)+len(ump62)))

cross: 23705, upgrade: 25666, ump: 22414 (all: 71785)
cross: 39932, upgrade: 41523, ump: 22638 (all: 104093)


In [362]:
seg_pd7_seg1 = seg_pd7_seg1.astype('str')
seg_pd7_seg2 = seg_pd7_seg2.astype('str')

In [363]:
cross71, upgrade71, ump71 = get_usecases(seg_pd7_seg1)
cross72, upgrade72, ump72 = get_usecases(seg_pd7_seg2)

In [364]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross71), len(upgrade71), len(ump71), 
              len(cross71)+len(upgrade71)+len(ump71)))
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross72), len(upgrade72), len(ump72), 
              len(cross72)+len(upgrade72)+len(ump72)))

cross: 2380, upgrade: 2589, ump: 2295 (all: 7264)
cross: 3972, upgrade: 4140, ump: 2312 (all: 10424)


In [365]:
seg_pd8_seg1 = seg_pd8_seg1.astype('str')
seg_pd8_seg2 = seg_pd8_seg2.astype('str')

In [366]:
cross81, upgrade81, ump81 = get_usecases(seg_pd8_seg1)
cross82, upgrade82, ump82 = get_usecases(seg_pd8_seg2)

In [367]:
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross81), len(upgrade81), len(ump81), 
              len(cross81)+len(upgrade81)+len(ump81)))
print('cross: {}, upgrade: {}, ump: {} (all: {})'
      .format(len(cross82), len(upgrade82), len(ump82), 
              len(cross82)+len(upgrade82)+len(ump82)))

cross: 14583, upgrade: 15562, ump: 13784 (all: 43929)
cross: 24423, upgrade: 25213, ump: 13921 (all: 63557)
